In [3]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from pocketbfast import PocketBfast
from datetime import datetime
import darts.timeseries as tseries
import dask.array as da
from nbeats_pytorch.model import NBeatsNet
import scipy.stats as stats
from pytorch_forecasting import  Baseline, NBeats, TimeSeriesDataSet
import torch
from torch import nn
import lightning.pytorch as pl
from lightning.pytorch.utilities.model_summary import ModelSummary
from typing import Dict
from pytorch_forecasting.models import BaseModel
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE, MASE, RMSE, MAE
import random
import itertools
from tqdm import tqdm
from lightning.pytorch.tuner import Tuner
from sklearn.model_selection import ParameterGrid
import sys
from concurrent.futures import ThreadPoolExecutor
import nbeats_training_script as nbt
import wandb
from pytorch_lightning.loggers import WandbLogger
# %env "WANDB_NOTEBOOK_NAME" "Nbeats Wandb Forecasting"
import warnings
warnings.filterwarnings('ignore')
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pablodlmora (vegetation-team). Use `wandb login --relogin` to force relogin


True

### Import the Time Series using the time collector
For this Section, it is important to Develop the creation of image time series, cleaned, masked, and set as dask arrays in order to make the script work. It was previously made relying on the proprietary libraries of LiveEO.

2024-02-13 11:13:11.990 | INFO     | liveeo_utils.database_utils.postgresql_db_utils:retrieve_geodf_for_sql_query:212 - Running query:
	SELECT * FROM veg_project_tiles.vitality_ger_kalman_borenkaefer_testing_train_tiles


Assembler object with root at /efs/pablo/time_experiments/borenkaefer/4b

In [ ]:
dates = [
     datetime.strptime(date, "%Y-%m-%d") for date in time_series.date_config["planet_PS"][::4]
 ]
gdf_labels = import_vector(file_path=labels_path)

### Splitting Tiles

These functions split the tiles in the AOI first into training and testing, and then split the training set of tiles into training and validation. The final sets purposefully leave out the tile 32_N1139_W11 to later test and use for exporting rasters. The random seed in both of the initial functions is optional and was done to get the same split in multiple runs.

In [6]:
def random_generator_iterator(seq, n, m):
    rand_seq = seq[:]
    random.seed(8)
    random.shuffle(rand_seq)
    limit = n-1
    for i,group in enumerate(zip(*([iter(rand_seq)]*m))):
        yield group
        if i == limit: break  



def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    m = int((len(lst)/n))
    sets = []
    for i in range(0, len(lst), m):
        sets.append(lst[i:i + m])
    np.random.seed(10)
    val_index = np.random.choice(10,2)
    val_set = np.concatenate((sets[val_index[0]],sets[val_index[1]]),axis=0)
    train_set = []
    for i in range(len(sets)):
        if i not in val_index:
            train_set.append(sets[i])
    training_set = np.vstack(train_set)
    # print(len(sets))
    # print('----')
    return val_set, training_set     


In [8]:
def tile_train_test_split(time_series):
    training_tiles, testing_tiles = random_generator_iterator(time_series.tiles, 2, 8)
    validation_set = []
    training_set = []
    testing_set = []
    testing_labels = []
    for tile in time_series:
        if tile.name in training_tiles:
            print(tile)
            tile_values = tile.values.compute()
            tile_labels = tile.labels.compute()
            
            val_set, train_set = chunks(tile_values,10)
            validation_set.append(val_set)
            training_set.append(train_set)
        
        else:
            testing_tile_values  = tile.values.compute()
            testing_tile_labels = tile.labels.compute()
            testing_set.append(testing_tile_values)
            testing_labels.append(testing_tile_labels)

    validation_set = np.vstack(validation_set)
    training_set = np.vstack(training_set)
    testing_set = np.vstack(testing_set)
    # testing_labels = np.vstack(testing_labels)


    return validation_set,training_set,testing_set

validation_set,training_set,testing_set = tile_train_test_split(time_series)

Tile 32_N1134_W18 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1134_W19 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1139_W13 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1139_W14 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1140_W11 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1140_W13 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1141_W12 stored at /efs/pablo/time_experiments/borenkaefer/4b
Tile 32_N1141_W14 stored at /efs/pablo/time_experiments/borenkaefer/4b


### Functions for the preprocessing of the data, to best work with it for NBEATS, and the Kalman filter

* Includes functions to average the time series to have a monthly cadence instead of weekly, and may generate errors if it is not the case. Also has interpolation to handle NaNs.

* Presets for a lot of future variables, which shouls be changed according to the shape of the dataset and the length of the time series. Forecast and Backcast lengths are important, and the backcast length always has to be longer than the forecast. Best is to have them do a 50/50 split, or as close to it as possible.

* Dictionaries for Hyperparameter tuning with Weights and Biases Library, as well as the set up for the Sweep

In [11]:
def map_indices(dates):
    start = dates[0]
    end = dates[-1]
    start = datetime(start.year, 1, 1)
    end = datetime(end.year, 12, 31)

    drange = pd.date_range(start, end, freq="d")
    ts = pd.Series(np.ones(len(dates)), dates)
    ts = ts.reindex(drange)
    indices = np.argwhere(~np.isnan(ts).to_numpy()).T[0]

    return indices
def average_time_series(array: np.ndarray, window_size: int = 4, mode: str = "mean"):
    """Aggregate the time step values by averaging them."""

    try:
        array = array.reshape(array.shape[0], array.shape[1], -1, window_size)
    except IndexError as exception:
        raise ValueError(
            f"The input time series length ({array.shape[1]}) can't be divided by"
            f" the window size ({window_size}). This can be worked around specifying"
            " a different number for the `from_time_step` parameter to make the"
            " division fit."
        ) from exception

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        reducer = da.nanmedian if mode == "median" else da.nanmean
        array = reducer(array, axis=3)
    
    return array

def interpolate_band(band: np.ndarray) -> np.ndarray:
    """Interpolate missing values in series."""
    band = band.copy()
    is_nan = np.isnan(band)
    nan_index = is_nan.nonzero()[0]
    value_index = (~is_nan).nonzero()[0]
    band[is_nan] = np.interp(nan_index, value_index, band[value_index])
    return band

In [12]:
sample_count = 16000   # Number of pixels to use.
validation_sample_count = 4000
data_scale = 10000  # Used to scale the values to [0, 1]
nodata_value = 32767
fill_value = 0  # Nodata points will be filled with this value.

# Data split config.
val_size = 270000
test_size = 135000

# Monitoring period.
forecast_length = 30
backcast_length = 30  

#Create time steps for Kalman and linear regression
tsteps_hist = np.linspace(1, 30.436875*backcast_length, backcast_length)
tsteps_monitoring = np.linspace(tsteps_hist[-1], 30.436875*forecast_length, forecast_length)

# Model config.
epochs = 10
bach_size = 128
output_dim = 1  # Keras n-beats supports multivariate modelling changing this param.

#For initial testing on a single pixel
pixel = 15000

sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

param_grid = {'epochs':{'values':[10,20]},
              'widths':{'values':[[32,256], [64,256], [32,512], [64,512], [32,1024],[64, 1024]]}, 
              'blocks':{'values':[[1,1],[2,2],[3,3]]},
              'layers':{'values':[[2,2],[3,3],[4,4]]},
              'batch_size':{'values':[128,256,512,1024]},
              'weight_decay':{'values':[1e-2,1e-3]}, 
              'sharing':{'values':[True,False]}}
sweep_config['parameters'] = param_grid

In [1]:
sweep_id = wandb.sweep(sweep_config, project="name of your wandb project")


NameError: name 'wandb' is not defined

### Preprocessing

Use of the previously written functions and split data to replace nans, split into the different bands, interpolate and calculate indices

In [14]:
training_set = training_set
validation_set = validation_set
testing_set = testing_set

data = time_series['32_N1139_W11'].values.compute()
data = data


# Replacing given NaN value with np.nan in the three sets
training_set = np.where(training_set == nodata_value, np.nan, training_set)
validation_set = np.where(validation_set == nodata_value, np.nan, validation_set)
testing_set = np.where(testing_set == nodata_value, np.nan, testing_set)
data = np.where(data == nodata_value, np.nan, data)


training_set = np.where(training_set <= 0, np.nan, training_set)
validation_set = np.where(validation_set <= 0, np.nan, validation_set)
testing_set = np.where(testing_set <= 0, np.nan, testing_set)
data = np.where(data <= 0, np.nan, data)


training_set = average_time_series(training_set)
validation_set = average_time_series(validation_set)
testing_set = average_time_series(testing_set)
data = average_time_series(data)


training_set = np.array(training_set)
validation_set = np.array(validation_set)
testing_set = np.array(testing_set)
data = np.array(data)


training_set = da.moveaxis(training_set, 0, 1)
validation_set = da.moveaxis(validation_set, 0, 1)
testing_set = da.moveaxis(testing_set, 0, 1)
data = da.moveaxis(data, 0, 1)


training_set = training_set / data_scale
validation_set = validation_set / data_scale
testing_set = testing_set / data_scale
data = data / data_scale


training_set[:,:,56] = np.nan
validation_set[:,:,56] = np.nan
testing_set[:,:,56] = np.nan
data[:,:,56] = np.nan



red_training = training_set[0,:,:]
nir_training = training_set[1,:,:]

red_validation = validation_set[0,:,:]
nir_validation = validation_set[1,:,:]

red_testing = testing_set[0,:,:]
nir_testing = testing_set[1,:,:]

red = data[0,:,:]
nir = data[1,:,:]

# greenI_training  = training_set[0,:,:]
# yellow_training = training_set[1,:,:]

# greenI_validation = validation_set[0,:,:]
# yellow_validation = validation_set[1,:,:]

# greenI_testing = testing_set[0,:,:]
# yellow_testing = testing_set[1,:,:]

# greenI = data[0,:,:]
# yellow = data[1,:,:]

# red = data[0,:,:]
# nir = data[1,:,:]


print(training_set.shape)
print(validation_set.shape)
print(testing_set.shape)

(2, 65393, 60)
(2, 16342, 60)
(2, 117474, 60)


In [15]:
def run_interpolate(dataset,red_band, nir_band):
    steps_red = []
    steps_nir = []
    
    for i in tqdm(range(dataset.shape[1])):
        try:
            steps_red.append(interpolate_band(red_band[i,:]))
            steps_nir.append(interpolate_band(nir_band[i,:]))
        except ValueError:
            steps_red.append(red_band[i,:])
            steps_nir.append(nir_band[i,:])
            
    red_band = np.row_stack(steps_red)  

    nir_band = np.row_stack(steps_nir)  

    return red_band, nir_band

In [16]:
red_training,nir_training = run_interpolate(training_set,red_training,nir_training)
red_validation,nir_validation = run_interpolate(validation_set,red_validation,nir_validation)
red_testing,nir_testing = run_interpolate(testing_set,red_testing,nir_testing)

# redge, nir = run_interpolate(data,redge,nir)

# greenI_training,yellow_training = run_interpolate(training_set,greenI_training,yellow_training)
# greenI_validation,yellow_validation = run_interpolate(validation_set,greenI_validation,yellow_validation)
# greenI_testing,yellow_testing = run_interpolate(testing_set,greenI_testing,yellow_testing)

# red,nir = run_interpolate(data,red,nir)

labels = time_series.labels_
labels = np.array(labels)
ndvi_training  =  (nir_training - red_training) / (nir_training + red_training)
ndvi_validation  =  (nir_validation - red_validation) / (nir_validation + red_validation)
ndvi_testing =  (nir_testing - red_testing) / (nir_testing + red_testing)
ndvi =  (nir - red) / (nir + red)

# ndre_training  =  (nir_training - redge_training) / (nir_training + redge_training)
# ndre_validation  =  (nir_validation - redge_validation) / (nir_validation + redge_validation)
# ndre_testing =  (nir_testing - redge_testing) / (nir_testing + redge_testing)
# ndre =  (nir - redge) / (nir + redge)

# training_set = yellow_training
# validation_set = yellow_validation
# testing_set = yellow_testing

# training_set = greenI_training
# validation_set = greenI_validation
# testing_set = greenI_testing

training_set = ndvi_training
validation_set = ndvi_validation
testing_set = ndvi_testing

# data = ndvi

# training_set = training_set[training_slice,:]
# validation_set = validation_set[validation_slice,:]
# testing_set = testing_set[testing_slice,:]


100%|██████████| 117474/117474 [00:01<00:00, 64309.35it/s]


### Function to create GeoDataframes for NBEATS Forecasting training and testing

In [18]:
def create_datasets(training_array,validation_array,testing_array,dates):    
    training_dataset = pd.DataFrame(training_array)
    training_dataset = training_dataset.transpose()
    # training_dataset = training_dataset.apply(lambda x: hampel(x,n_sigma=2.0).filtered_data)
    training_dataset['dates'] = dates
    training_dataset.set_index('dates',append=True,inplace=True)
    training_dataset = training_dataset.transpose()
    training_dataset = training_dataset.stack(list(range(training_dataset.columns.nlevels)))
    training_dataset = pd.DataFrame(training_dataset)
    training_dataset= training_dataset.rename(columns = {0:'values'})
    training_dataset.index.names = ['group','time_idx','dates']
    training_dataset = training_dataset.reset_index()

    validation_dataset = pd.DataFrame(validation_array)
    validation_dataset = validation_dataset.transpose()
    # validation_dataset = validation_dataset.apply(lambda x: hampel(x,n_sigma=2.0).filtered_data)
    validation_dataset['dates'] = dates
    validation_dataset.set_index('dates',append=True,inplace=True)
    validation_dataset = validation_dataset.transpose()
    validation_dataset = validation_dataset.stack(list(range(validation_dataset.columns.nlevels)))
    validation_dataset = pd.DataFrame(validation_dataset)
    validation_dataset= validation_dataset.rename(columns = {0:'values'})
    validation_dataset.index.names = ['group','time_idx','dates']
    validation_dataset = validation_dataset.reset_index()

    testing_dataset = pd.DataFrame(testing_array)
    testing_dataset = testing_dataset.transpose()
    # testing_dataset = testing_dataset.apply(lambda x: hampel(x,n_sigma=2.0).filtered_data)
    # dataset_testing = testing_hampel.filtered_data
    testing_dataset['dates'] = dates
    testing_dataset.set_index('dates',append=True,inplace=True)
    testing_dataset = testing_dataset.transpose()
    testing_dataset = testing_dataset.stack(list(range(testing_dataset.columns.nlevels)))
    testing_dataset = pd.DataFrame(testing_dataset)
    testing_dataset = testing_dataset.rename(columns = {0:'values'})
    testing_dataset.index.names = ['group','time_idx','dates']
    testing_dataset = testing_dataset.reset_index()

    return training_dataset,validation_dataset,testing_dataset

### Function to train model and do hyperparameter tuning with wandb

Command to perform sweep, and then save the best metrics

In [20]:
# def train_fit_model(config=None):
     
#     training_dataset,validation_dataset,testing_dataset = create_datasets(training_set,validation_set,testing_set,dates[3:])
#     # create dataset and dataloaders
#     max_encoder_length = backcast_length
#     max_prediction_length = forecast_length

#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#     training_cutoff = training_dataset["time_idx"].max() - max_prediction_length

#     context_length = max_encoder_length
#     prediction_length = max_prediction_length


#     training = TimeSeriesDataSet(
#     training_dataset,
#     time_idx="time_idx",
#     target="values",
#     categorical_encoders={"group": NaNLabelEncoder().fit(training_dataset.group)},
#     group_ids=["group"],
#     # only unknown variable is "value" - and N-Beats can also not take any additional variables
#     time_varying_unknown_reals=["values"],
#     max_encoder_length=context_length,
#     max_prediction_length=prediction_length,
#     allow_missing_timesteps=True
# )
    
#     # checkpoint_callback = ModelCheckpoint(monitor='val_accuracy', mode='max')

#     with wandb.init(config=config):
#         wandb_logger = WandbLogger(project='nbeats-wand-forecasting', log_model='all')
#         config = wandb.config

#         train_dataloader = training.to_dataloader(train=True, batch_size=config.batch_size, num_workers=0)
#         validation = TimeSeriesDataSet.from_dataset(training, validation_dataset, min_prediction_idx=training_cutoff + 1)
#         val_dataloader = validation.to_dataloader(train=False, batch_size=config.batch_size, num_workers=0)

        
#         # calculate baseline absolute error
#         actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).cuda()
#         baseline_predictions = Baseline().predict(val_dataloader).cuda()
#         SMAPE()(baseline_predictions, actuals)
       
#         baseline = SMAPE()(baseline_predictions, actuals)
#         wandb.log({"Baseline SMAPE": baseline})

#         pl.seed_everything(42)
#         trainer = pl.Trainer(accelerator="gpu", gradient_clip_val=0.01,devices=1,logger=wandb_logger)
#         net = NBeats.from_dataset(training, learning_rate=3e-2, weight_decay=config.weight_decay, widths=config.widths, backcast_loss_ratio=0.1)

#         # find optimal learning rate
#         res = Tuner(trainer).lr_find(net, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
#         print(f"suggested learning rate: {res.suggestion()}")
#         # fig = res.plot(show=True, suggest=True)
#         # fig.show()
#         net.hparams.learning_rate = res.suggestion()
#         lr_suggested = net.hparams.learning_rate

#         print(lr_suggested)
#         #Fitting Model
#         early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
#         trainer = pl.Trainer(
#             max_epochs=config.epochs,
#             accelerator="gpu",
#             enable_model_summary=True,
#             gradient_clip_val=0.01,
#             callbacks=[early_stop_callback],
#             limit_train_batches=150,
#             logger=wandb_logger
#         )
        
        

#         net = NBeats.from_dataset(
#             training,
#             learning_rate=lr_suggested,
#             num_blocks = config.blocks,
#             num_block_layers =config.layers,
#             # log_interval=10,
#             # log_val_interval=1,
#             weight_decay=config.weight_decay,
#             widths=config.widths,
#             sharing = config.sharing,
#             backcast_loss_ratio=1.0,
            
#         ) 
        
#         net.to(device)
        
#         # wandb_logger.watch(net, log_graph=False)

#         trainer.fit(
#             net,
#             train_dataloaders=train_dataloader,
#             val_dataloaders=val_dataloader,
           
#         )
        

#         #Validation error calculation
#         best_model_path = trainer.checkpoint_callback.best_model_path
#         best_model = NBeats.load_from_checkpoint(best_model_path)

#         actuals_validation = torch.cat([y[0] for x, y in iter(val_dataloader)]).cuda()
#         predictions_validation = best_model.predict(val_dataloader).cuda()
        
#         smape_validation = SMAPE()(predictions_validation,actuals_validation).detach().cpu().numpy()
#         rmse_validation = RMSE()(predictions_validation,actuals_validation).detach().cpu().numpy()

#         error_validation = (actuals_validation - predictions_validation).abs().mean()

#         #Predictions
#         testing = TimeSeriesDataSet.from_dataset(training, testing_dataset, min_prediction_idx=training_cutoff + 1)
#         test_dataloader = testing.to_dataloader(train=False, batch_size=config.batch_size, num_workers=0)


#         actuals = torch.cat([y[0] for x, y in iter(test_dataloader)]).cuda()
#         predictions = best_model.predict(test_dataloader).cuda()
       
#         smape = SMAPE()(predictions,actuals).detach().cpu().numpy()
#         rmse = RMSE()(predictions,actuals).detach().cpu().numpy()
#         # (actuals - predictions).abs().mean()
       
#         items = best_model.predict(test_dataloader, mode='raw', return_x=True)
#         x = items.x
#         raw_predictions = items[0]
#         nb_predictions_testing = raw_predictions.prediction.detach().cpu().numpy()
        
    
#         wandb.log({ "SMAPE": smape, 'RMSE':rmse})

#         return nb_predictions_testing




In [21]:
# wandb.agent(sweep_id, train_fit_model, count=10)

In [23]:
api = wandb.Api()

# run is specified by <entity>/<project>/<run_id>
run = api.run("vegetation-team/nbeats-wand-forecasting/2jknx9e0")

# save the metrics for the run to a csv file
metrics_dataframe = run.history()
metrics_dataframe.to_csv("/home/ubuntu/liveeo_modules/pablo_lab/pablo_lab/Results/wandb_boren_ndvi_metrics.csv")

### Cells to train single model on the best hyperparameters found through optimization

Followed by cell to fit model on the testing tile to save the predictions and the model parameters.

In [24]:
# create dataset and dataloaders
training_dataset,validation_dataset,testing_dataset = create_datasets(training_set,validation_set,testing_set,dates)
max_encoder_length = backcast_length
max_prediction_length = forecast_length

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


training_cutoff = training_dataset["time_idx"].max() - max_prediction_length

context_length = max_encoder_length
prediction_length = max_prediction_length


training = TimeSeriesDataSet(
training_dataset,
time_idx="time_idx",
target="values",
categorical_encoders={"group": NaNLabelEncoder().fit(training_dataset.group)},
group_ids=["group"],
# only unknown variable is "value" - and N-Beats can also not take any additional variables
time_varying_unknown_reals=["values"],
max_encoder_length=context_length,
max_prediction_length=prediction_length,
allow_missing_timesteps=True
)


train_dataloader = training.to_dataloader(train=True, batch_size=512, num_workers=0)
validation = TimeSeriesDataSet.from_dataset(training, validation_dataset, min_prediction_idx=training_cutoff + 1)
val_dataloader = validation.to_dataloader(train=False, batch_size=512, num_workers=0)

# calculate baseline absolute error
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).cuda()
baseline_predictions = Baseline().predict(val_dataloader).cuda()
SMAPE()(baseline_predictions, actuals)

pl.seed_everything(42)
trainer = pl.Trainer(accelerator="auto", gradient_clip_val=0.01,devices=1)
net = NBeats.from_dataset(training, learning_rate=3e-2, weight_decay=1e-2, widths=[32, 512], backcast_loss_ratio=0.1)

# find optimal learning rate
res = Tuner(trainer).lr_find(net, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
print(f"suggested learning rate: {res.suggestion()}")
# fig = res.plot(show=True, suggest=True)
# fig.show()
net.hparams.learning_rate = res.suggestion()
lr_suggested = net.hparams.learning_rate


#Fitting Model
early_stop_callback = EarlyStopping(monitor="val_SMAPE", min_delta=1e-4, patience=10, verbose=False, mode="min")
trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    enable_model_summary=True,
    gradient_clip_val=0.01,
    callbacks=[early_stop_callback],
    limit_train_batches=150
)

net = NBeats.from_dataset(
    training,
    learning_rate=lr_suggested,
    # num_blocks = [3],
    # num_block_layers = [3],
    log_interval=-1,
    log_val_interval=1,
    weight_decay=0.01,
    widths=[64,256],
    sharing = False,
    backcast_loss_ratio=1.0
) 

net.to(device)

trainer.fit(
    net,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

#Predictions
testing = TimeSeriesDataSet.from_dataset(training, testing_dataset, min_prediction_idx=training_cutoff + 1)
test_dataloader = testing.to_dataloader(train=False, batch_size=512, num_workers=0)


best_model_path = trainer.checkpoint_callback.best_model_path
best_model = NBeats.load_from_checkpoint(best_model_path)

actuals = torch.cat([y[0] for x, y in iter(test_dataloader)]).cuda()
predictions = best_model.predict(test_dataloader).cuda()

smape = SMAPE()(predictions,actuals)

rmse = RMSE()(predictions,actuals)
error_testing =  (actuals - predictions).abs().mean()

items = best_model.predict(test_dataloader, mode='raw', return_x=True)
x = items.x
raw_predictions = items[0]
nb_predictions_boren = raw_predictions.prediction.detach().cpu().numpy()

2024-02-13 11:14:05.992689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 11:14:06.727881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.05011872336272722
Restoring states from the checkpoint path at /home/ubuntu/liveeo_modules/pablo_lab/pablo_lab/.lr_find_84b4a8a3-e3ab-411d-92fd-bdf0908419ba.ckpt
Restored all states from the checkpoint at /home/ubuntu/liveeo_modules/pablo_lab/pablo_lab/.lr_find_84b4a8a3-e3ab-411d-92fd-bdf0908419ba.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | MASE       | 0     
1 | logging_metrics | ModuleList | 0     
2 | net_blocks      | ModuleList | 473 K 
-----------------------------------------------
473 K     Trainable params
0         Non-trainable params
473 K     Total params
1.892     Total estimated model params size (MB)


suggested learning rate: 0.05011872336272722


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [25]:
# #Fitting Model
# early_stop_callback = EarlyStopping(monitor="val_SMAPE", min_delta=1e-4, patience=10, verbose=False, mode="min")
# trainer = pl.Trainer(
#     max_epochs=20,
#     accelerator="gpu",
#     enable_model_summary=True,
#     gradient_clip_val=0.01,
#     callbacks=[early_stop_callback],
#     limit_train_batches=150
# )

# net = NBeats.from_dataset(
#     training,
#     learning_rate=lr_suggested,
#     # num_blocks = [3],
#     # num_block_layers = [3],
#     log_interval=-1,
#     log_val_interval=1,
#     weight_decay=0.001,
#     widths=[32,1024],
#     sharing = False,
#     backcast_loss_ratio=1.0
# ) 

# best_model_path = trainer.checkpoint_callback.best_model_path
# best_model = NBeats.load_from_checkpoint(best_model_path)

# testing_32_N1139_W11 = nbt.create_single_dataset(ndre,dates[3:])
# testing = TimeSeriesDataSet.from_dataset(training, testing_dataset, min_prediction_idx=training_cutoff + 1)
# test_dataloader = testing.to_dataloader(train=False, batch_size=1024, num_workers=0)


# best_model_path = trainer.checkpoint_callback.best_model_path
# best_model = NBeats.load_from_checkpoint(best_model_path)

# actuals = torch.cat([y[0] for x, y in iter(test_dataloader)]).cuda()
# predictions = best_model.predict(test_dataloader).cuda()

# smape = SMAPE()(predictions,actuals)

# rmse = RMSE()(predictions,actuals)
# error_testing =  (actuals - predictions).abs().mean()

# items = best_model.predict(test_dataloader, mode='raw', return_x=True)
# x = items.x
# raw_predictions = items[0]
# nb_predictions_redge_boren = raw_predictions.prediction.detach().cpu().numpy()

In [26]:
torch.save(best_model.state_dict(),'/home/ubuntu/liveeo_modules/pablo_lab/pablo_lab/Results/nbeats_boren_ndvi_model.pt')

In [27]:
nbeats_predictions = pd.DataFrame(nb_predictions_boren)
nbeats_predictions.to_csv('/home/ubuntu/liveeo_modules/pablo_lab/pablo_lab/Results/nbeats_predictions_ndvi_testing.csv')